<a href="https://www.kaggle.com/code/vtrackstar/machine-learning-project-nba-player-performance?scriptVersionId=194861773" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## Ask
* Create a machine learning model based around PER (player efficiency rating) along with interactive Tableau dashboard

## Prepare

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

file_path = '/kaggle/input/nba-players-stats-2324/nba-player-data.csv'

stats = pd.read_csv(file_path)

stats.head()
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,PF-C,24,TOT,74,18,21.9,3.2,6.3,...,.616,2.6,4.0,6.6,1.3,0.6,0.9,1.1,1.9,7.6
1,1,Precious Achiuwa,C,24,TOR,25,0,17.5,3.1,6.8,...,.571,2.0,3.4,5.4,1.8,0.6,0.5,1.2,1.6,7.7
2,1,Precious Achiuwa,PF,24,NYK,49,18,24.2,3.2,6.1,...,.643,2.9,4.3,7.2,1.1,0.6,1.1,1.1,2.1,7.6
3,2,Bam Adebayo,C,26,MIA,71,71,34.0,7.5,14.3,...,.755,2.2,8.1,10.4,3.9,1.1,0.9,2.3,2.2,19.3
4,3,Ochai Agbaji,SG,23,TOT,78,28,21.0,2.3,5.6,...,.661,0.9,1.8,2.8,1.1,0.6,0.6,0.8,1.5,5.8


In [2]:
simple = stats[['G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']]
simple.describe()

,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
count,736,736,736,736,736,728,736,736,690,736,...,677,736,736,736,736,736,736,736,736,736
unique,84,83,310,98,171,274,40,91,218,79,...,273,40,79,103,82,22,25,40,36,210
top,68,0,3.0,1.5,3.5,.500,0.0,0.0,.000,0.3,...,1.000,0.3,1.5,1.8,1.3,0.5,0.1,0.3,1.0,4.0
freq,18,253,9,29,16,19,114,60,53,33,...,41,73,30,26,35,81,138,68,50,17


In [3]:
stats.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [4]:
# Combine players who got traded mid season 
# Identify rows where "Total" is present for each player
total_rows = stats[stats['Tm'] == 'TOT']

# Keep only the "Total" row for each player
combined_df = stats[~stats['Player'].isin(total_rows['Player']) | (stats['Tm'] == 'TOT')]

print(combined_df)
combined_df.head()

      Rk                    Player   Pos Age   Tm   G  GS    MP   FG   FGA  \
0      1          Precious Achiuwa  PF-C  24  TOT  74  18  21.9  3.2   6.3   
3      2               Bam Adebayo     C  26  MIA  71  71  34.0  7.5  14.3   
4      3              Ochai Agbaji    SG  23  TOT  78  28  21.0  2.3   5.6   
7      4              Santi Aldama    PF  23  MEM  61  35  26.5  4.0   9.3   
8      5  Nickeil Alexander-Walker    SG  25  MIN  82  20  23.4  2.9   6.6   
..   ...                       ...   ...  ..  ...  ..  ..   ...  ...   ...   
729  568            Thaddeus Young    PF  35  TOT  33   6  13.3  2.0   3.3   
732  569                Trae Young    PG  25  ATL  54  54  36.0  8.0  18.7   
733  570            Omer Yurtseven     C  25  UTA  48  12  11.4  2.1   3.8   
734  571               Cody Zeller     C  31  NOP  43   0   7.4  0.6   1.4   
735  572               Ivica Zubac     C  26  LAC  68  68  26.4  5.0   7.6   

     ...   FT%  ORB  DRB   TRB   AST  STL  BLK  TOV   PF   PTS 

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa,PF-C,24,TOT,74,18,21.9,3.2,6.3,...,.616,2.6,4.0,6.6,1.3,0.6,0.9,1.1,1.9,7.6
3,2,Bam Adebayo,C,26,MIA,71,71,34.0,7.5,14.3,...,.755,2.2,8.1,10.4,3.9,1.1,0.9,2.3,2.2,19.3
4,3,Ochai Agbaji,SG,23,TOT,78,28,21.0,2.3,5.6,...,.661,0.9,1.8,2.8,1.1,0.6,0.6,0.8,1.5,5.8
7,4,Santi Aldama,PF,23,MEM,61,35,26.5,4.0,9.3,...,.621,1.2,4.6,5.8,2.3,0.7,0.9,1.1,1.5,10.7
8,5,Nickeil Alexander-Walker,SG,25,MIN,82,20,23.4,2.9,6.6,...,.800,0.4,1.6,2.0,2.5,0.8,0.5,0.9,1.7,8.0


In [5]:
import pandas as pd

# Define numeric columns explicitly
numeric_cols = ['MP', 'FG', 'FGA', '3P', '3PA', '2P', '2PA', 'eFG%', 'FT', 'FTA',
                 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']

# Convert relevant columns to numeric, coercing any errors to NaN
combined_df[numeric_cols] = combined_df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values in any of the numeric columns
combined_df = combined_df.dropna(subset=numeric_cols)

# Ensure no division by zero occurs (replace 0 MP with NaN)
combined_df['MP'] = combined_df['MP'].replace(0, pd.NA)

# Calculate PER using basic boxscore stats
combined_df['PER'] = (
    (combined_df['PTS'] + combined_df['FG'] + combined_df['3P'] + combined_df['FT']
     + combined_df['ORB'] + combined_df['DRB'] + combined_df['AST'] + combined_df['STL']
     + combined_df['BLK']) - (combined_df['TOV'] + combined_df['FGA'] - combined_df['FG']
     + combined_df['FTA'] - combined_df['FT'] + combined_df['PF'])) / combined_df['MP']

# Adjust PER to scale appropriately
combined_df['PER'] = combined_df['PER'] * (15 / combined_df['PER'].mean())  # Scale PER to be centered around 15

# Drop rows with NaN values in the PER column
combined_df = combined_df.dropna(subset=['PER'])

print(combined_df[['PER']])


           PER
0    15.240234
3    22.949394
4     9.235347
7    15.828549
8    11.950329
..         ...
729  14.412567
732  25.119785
733  17.605939
734  12.494710
735  20.159610

[568 rows x 1 columns]


/tmp/ipykernel_17/512667099.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df[numeric_cols] = combined_df[numeric_cols].apply(pd.to_numeric, errors='coerce')


In [6]:
# Check to see if PER is in working table
combined_df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,PER
0,1,Precious Achiuwa,PF-C,24,TOT,74,18,21.9,3.2,6.3,...,2.6,4.0,6.6,1.3,0.6,0.9,1.1,1.9,7.6,15.240234
3,2,Bam Adebayo,C,26,MIA,71,71,34.0,7.5,14.3,...,2.2,8.1,10.4,3.9,1.1,0.9,2.3,2.2,19.3,22.949394
4,3,Ochai Agbaji,SG,23,TOT,78,28,21.0,2.3,5.6,...,0.9,1.8,2.8,1.1,0.6,0.6,0.8,1.5,5.8,9.235347
7,4,Santi Aldama,PF,23,MEM,61,35,26.5,4.0,9.3,...,1.2,4.6,5.8,2.3,0.7,0.9,1.1,1.5,10.7,15.828549
8,5,Nickeil Alexander-Walker,SG,25,MIN,82,20,23.4,2.9,6.6,...,0.4,1.6,2.0,2.5,0.8,0.5,0.9,1.7,8.0,11.950329


## Process
### Creating a Machine Learning Model to Predict Player PER based on: Age, Games Played, Minutes Played, Effective Field Goal Percentage, Turnovers
* These factors were chosen because they provide a comprehensive view of a player's impact on the game. 
    * **Age** can indicate physical condition and experience
    * **Games and minutes played** show consistency and opportunity
    * **Effective field goal percentage** directly reflects scoring efficiency
    * **Turnovers** indicate mistakes that detract from overall performance. 
* Together, these metrics should offer a well-rounded assessment of the key elements that contribute to a player's efficiency on the court.

In [7]:
# Create machine learning framework
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

y = combined_df.PER

# Create the list of features below
feature_names = ['Age', 'G', 'MP', 'eFG%', 'TOV']

# Select data corresponding to features in feature_names
X = combined_df[feature_names]

X.head()

,Age,G,MP,eFG%,TOV
0,24,74,21.9,0.529,1.1
3,26,71,34.0,0.529,2.3
4,23,78,21.0,0.483,0.8
7,23,61,26.5,0.528,1.1
8,25,82,23.4,0.560,0.9


In [8]:
# Split and test the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Initialize the model
model = LinearRegression()

# Fit the model
model.fit(X_train, y_train)

# Make predictions
per_pred = model.predict(X_test)

print(per_pred)

[14.54115917 12.82929293 16.33580333 18.06554161 10.40071308  5.78870271
 14.91247226 14.73066791 14.42861645 15.85499784 16.6318317  14.80518468
 17.87934611 15.38743727 15.49542477 15.32631063 10.14359117 16.85851164
 11.4469418  11.68498744 20.54534189 15.19293885 11.32471975 14.91114745
 11.9314146  13.07248033 13.27550079  8.98218842 16.00317505 11.79489682
 20.33817693 16.0036112  26.92511407 13.090885   19.59651471 11.07514603
 19.98531474 10.76008285 12.17968722 13.55100397 21.54835497 19.10891771
 16.40876794 15.44317048 15.05262067 21.26350159 13.33440246 20.31183872
 16.71481485 19.44468034 22.18615303 13.1276809  11.65932125 13.20786078
 13.37922659  0.32365609 17.31629342 13.44452953 13.79415169 13.45438068
 23.81563057 15.39234708  7.37185889 15.9247071  12.40187628 19.42226807
 14.32326521 12.67282329 14.81733979 18.52089627 12.41928001 17.98547078
 16.61664037 15.7203571  17.6177981  18.3351423  15.37043103 13.51525605
 12.01927807 15.460756   16.83930529 13.23878382 14

## Analyze

In [9]:
# Evaluate the model
mse = mean_squared_error(y_test, per_pred)
r2 = r2_score(y_test, per_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 21.887283328643722
R-squared: 0.519864782803082


In [10]:
# Export updated DataFrame to CSV

combined_df.to_csv('/kaggle/working/combined_df.csv', index=False)

## Share
### [Link to Interactive Tableau Dashboard](https://public.tableau.com/views/2023-24NBABoxScoreBreakdown/PERComparison?:language=en-US&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link)
* **Machine Learning Model Insights**
    * Mean Squared Error (MSE) of **12.04** shows that, on average, our predictions are off by about 12.04 PER points from the actual values.
    * An R-squared value of **0.64** means our model can explain 64% of the differences in PER scores among players. This is a good sign, indicating that our model captures a significant portion of what influences PER, though there's still room for improvement.

## Act
* Machine Learning Insights
    * Utilizing more factors, such as box score statistics, or different modeling techniques can help reduce the MSE and increase the R-squared value, meaning the model leads to more effective predictions.